In [155]:
import pandas as pd
from gurobipy import *
import openpyxl

In [156]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("input.xlsx")
main = wb['original']

In [157]:
# read objevtive
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')
# read obj coefficient
obj_coeff = []
for c in range(2,10000):
    v = main.cell(column=c, row=3).value
    if v is None:
        break
    else:
        obj_coeff.append(v)
n_variables = len(obj_coeff)

#read variable properties
variable_name = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=2).value
    if v is None:
        variable_name.append('x'+str(c+1))
    elif type(v) != str:
        variable_name.append(str(v))
    else:
        variable_name.append(v)
        
variable_type = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=4).value
    if v is None:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)
    
variable_lb = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=5).value
    if v is None:
        t = 0
    elif v == '"+inf"':
        raise NameError('Variable lower bound cannot be +inf')
    elif v == '"-inf"':
        t = -GRB.INFINITY
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable lower bound cannot be recognized')
    variable_lb.append(t)

variable_ub = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=6).value
    if v is None:
        t = GRB.INFINITY
    elif v == '"+inf"':
        t = GRB.INFINITY
    elif v == '"-inf"':
        raise NameError('Variable upper bound cannot be -inf')
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable upper bound cannot be recognized')
    variable_ub.append(t)
print(obj_coeff)

[1000, -350, -53.5, -4, -4, -4, -4, -6, -6, -6, -6]


In [158]:
# read constraints
## find constraint column index
for c in range(1, n_variables+100):
    v = main.cell(column=c, row=8).value
    if v == 'constraint type':
        const_typ_cind = c
        break

if const_typ_cind-2 != n_variables:
    raise NameError('Number of variables does not match the constraints')
    
## find last constraint row index
for r in range(9, 100000):
    v = main.cell(column=const_typ_cind, row=r).value
    if v is None:
        break
    elif v not in ["<=", ">=", "="]:
        raise NameError("Wrong constraint type")
    else:
        last_r_consraints=r

## read constraints as dataframe
for r in range(9, last_r_consraints+1):
    v = main.cell(column=const_typ_cind+2, row=r).value
    if v is None:
        main.cell(column=const_typ_cind+2, row=r).value = "c"+str(r-8)
    elif type(v) != str:
        main.cell(column=const_typ_cind+2, row=r).value = str(v)
    else:
        1==1        
values = []
cons_df = pd.DataFrame()
for r in range(9, last_r_consraints+1):
    r_values = []
    for c in range(2, const_typ_cind+3):
        r_values.append(main.cell(column=c, row=r).value)
    values.append(r_values)
    cons_df = pd.DataFrame(values)
    cons_df.fillna(0, inplace=True)
    cons_df.rename(columns={const_typ_cind-2:'constraint type', 
                            const_typ_cind-1:'RHS values', 
                            const_typ_cind:'constraint name'}, inplace=True)

In [159]:
# setup model
m = Model()

In [160]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
    x[i].setAttr('lb', variable_lb[i])
    x[i].setAttr('ub', variable_ub[i])

In [161]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [162]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['constraint type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i], 
                    name = le_const['constraint name'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['constraint type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i], 
                    name = ge_const['constraint name'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['constraint type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i], 
                    name = eq_const['constraint name'][i])

In [163]:
## run the model
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 11 columns and 26 nonzeros
Model fingerprint: 0x8ce74bdd
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [4e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.03s
Presolved: 6 rows, 11 columns, 17 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8868851e+07   2.358606e+03   0.000000e+00      0s
       6    1.3155223e+07   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.06 seconds (0.00 work units)
Optimal objective  1.315522275e+07


In [164]:
# print optimal solution
m.printAttr('X')


    Variable            X 
-------------------------
      demand      18562.5 
  production        14850 
           u       3712.5 
          r1          320 
          r2          480 
          r3          640 
          r4          480 
          v1        51.25 
          v2      142.875 
          v3      205.625 
          v4          180 


In [165]:
# sensitivity analysis
for v in m.getVars():
    print("For Variable " + v.VarName+ ":")
    print("Minimum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjLow))
    print("Maximum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjUp))

For Variable demand:
Minimum value coefficient can take before the optimal decision changes is 291.5079999999999
Maximum value coefficient can take before the optimal decision changes is inf
For Variable production:
Minimum value coefficient can take before the optimal decision changes is -1235.615
Maximum value coefficient can take before the optimal decision changes is inf
For Variable u:
Minimum value coefficient can take before the optimal decision changes is -1000.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable r1:
Minimum value coefficient can take before the optimal decision changes is -6.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable r2:
Minimum value coefficient can take before the optimal decision changes is -6.0
Maximum value coefficient can take before the optimal decision changes is inf
For Variable r3:
Minimum value coefficient can take before the optimal decision changes is -6.0
Ma

In [166]:
for c in m.getConstrs():
    print("For constraint " + c.ConstrName+ ":")
    print("Shawdow Price is " + str(c.pi))
    print("Minimum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSLow))
    print("Maximum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSUp))

For constraint 2:
Shawdow Price is 1183.125
Minimum value RHS can take before the shadow price changes is -2970.0
Maximum value RHS can take before the shadow price changes is 1150.0
For constraint 3:
Shawdow Price is 2.0
Minimum value RHS can take before the shadow price changes is 251.25
Maximum value RHS can take before the shadow price changes is 371.25
For constraint 4:
Shawdow Price is 2.0
Minimum value RHS can take before the shadow price changes is 442.875
Maximum value RHS can take before the shadow price changes is 622.875
For constraint 5:
Shawdow Price is 2.0
Minimum value RHS can take before the shadow price changes is 605.625
Maximum value RHS can take before the shadow price changes is 845.625
For constraint 6:
Shawdow Price is 19928.3375
Minimum value RHS can take before the shadow price changes is 388.8888888888889
Maximum value RHS can take before the shadow price changes is 506.82926829268297
For constraint 7:
Shawdow Price is 0.0
Minimum value RHS can take before th